In [84]:
#1. given four coordinates and a rotation angle, get XatTopEdge and YatLeftEdge.
chirop = pd.read_pickle('../data/1C/10 1C/0m_4m.pkl')
tl, 
#2. add x to top edge and y at left edge to get the top-left chiroptera point.

#3. calculate the reset between the chiroptera and atl, and add it to atl. 

#4. add the rotated offset to the atl.

#5. run correlation!


In [38]:
import numpy as np
1/np.cos(np.deg2rad(17.9)) * 1000

1050.867886489478

In [109]:
import pandas as pd
import numpy as np
import math

#Because the image matcher matches ATL and chiroptera images from the top left,
#it is necessary to geolocate the original top left corner of the Chiroptera.
#To do this, we unrotate the original Chiroptera LiDAR (which is at an angle), gets (xmin, ymax), and re-rotate it.

def rotate(origin, point, angle):
    ox, oy = origin
    px, py = point

    qx = ox + math.cos(angle) * (px - ox) - math.sin(angle) * (py - oy)
    qy = oy + math.sin(angle) * (px - ox) + math.cos(angle) * (py - oy)
    return qx, qy

b1, b2 = 0,4
df = pd.read_pickle(f'../data/1C/10 1C/{b1}m_{b2}m.pkl')
ANGLE = np.deg2rad(17.9)
center = ((min(df['x']) + max(df['x'])) / 2, (min(df['y']) + max(df['y'])) / 2)

for index, row in df.iterrows():
    (x,y) = rotate(center, (row['x'], row['y']), ANGLE)
    row['x'], row['y'] = (x,y)

(rx, ry) = (min(df['x']), max(df['y']))
(urx, ury) = rotate(center, (rx, ry), -ANGLE)
#df.to_csv('rot_debug_chirop_df.csv', index=False)
print((urx, ury))

(570762.9061543589, 9497299.94695935)


In [ ]:
#Top left results:
#17.88: 570762.38, 9497300.03
#17.9: 570762.90, 9497299.95

In [107]:
import math

import pandas as pd
import geopandas as gpd
chirop = pd.read_pickle('../data/1C/10 1C/0m_4m.pkl')
atl_df = pd.read_csv('unshifted_gt3r_full.csv')

b1 = 0
b2 = b1 + 4
chirop_df = pd.read_pickle(f'../data/1C/10 1C/{b1}m_{b2}m.pkl')

x_min = min(chirop_df['x'])
x_max = max(chirop_df['x'])
y_min = min(chirop_df['y'])
y_max = max(chirop_df['y'])

print(f"xrange: {x_max - x_min} yrange: {y_max - y_min}")
#getting the correct cutoffs here is critical for the shift, because we base our shift on the top-left point.
#Why are the correct bounds around 1737:3737? We would expect it to be 2000:4000
#the original df has bounds 2000:4000 above the y_min

#SET ATL BOUNDS
LBOUND = 1737
HBOUND = LBOUND + 2000
atl_df = atl_df[(atl_df['y'] >= y_min + LBOUND) & (atl_df['y'] <= y_min + HBOUND)]
print(f'{x_min} {x_max} {y_min} {y_max}')
TL_POINT = (570762.9061543589, 9497299.94695935)
x_max = max(atl_df['x'])
y_max = max(atl_df['y'])
offset = (TL_POINT[0] - x_max, TL_POINT[1] - y_max)
atl_df['x'] = atl_df['x'] + offset[0]
atl_df['y'] = atl_df['y'] + offset[1]

#At this point, the ATL has been matched to the top-left corner.
#SHIFT WITH BEST FIT (from image_matcher)
ANGLE = 17.88
def rotate(origin, point, angle):
    ox, oy = origin
    px, py = point

    qx = ox + math.cos(angle) * (px - ox) - math.sin(angle) * (py - oy)
    qy = oy + math.sin(angle) * (px - ox) + math.cos(angle) * (py - oy)
    return qx, qy

#(x,y)
best_fit = (35, 890)
best_fit = rotate((0,0), best_fit, np.deg2rad(ANGLE))
print(best_fit)
#the calculated y shift is in the negative direction, so we reverse it here.
atl_df['x'] = atl_df['x'] + best_fit[0]
atl_df['y'] = atl_df['y'] - best_fit[1]

atl_df.to_csv('debug_shifted_atl.csv', index=False)

xrange: 1761.9299999999348 yrange: 4503.140000000596
569358.0299999999 571119.9599999998 9492739.940000001 9497243.080000002
(-239.94218549855844, 857.760308954766)
